In [ ]:
# !pip install transformers

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import gc, warnings, random, time, os

from pathlib import Path

from tqdm.notebook import tqdm

warnings.filterwarnings('ignore')

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.optim import Adam, lr_scheduler
from torch.utils.data import Dataset, DataLoader
from transformers import AdamW
from transformers import AutoModel, AutoTokenizer, AutoConfig
from transformers import get_cosine_schedule_with_warmup

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold

import seaborn as sns

import gc
gc.enable()

### Folders and Dataframes

In [2]:
DATA_PATH = Path('/home/commonlit/data/')
assert DATA_PATH.exists()
MODELS_PATH = Path('/home/commonlit/models/')
if not MODELS_PATH.exists():
    os.mkdir(MODELS_PATH)
assert MODELS_PATH.exists()

In [3]:
train_df = pd.read_csv(DATA_PATH/'train-orig.csv')
test_df = pd.read_csv(DATA_PATH/'test.csv')
sample_df = pd.read_csv(DATA_PATH/'sample_submission.csv')

In [4]:
def remove_unnecessary(df):
    df.drop(df[df['target'] == 0].index, inplace=True)
    df.reset_index(drop=True, inplace=True)
    
remove_unnecessary(train_df)

In [5]:
train_df

,id,url_legal,license,excerpt,target,standard_error
0,c12129c31,NaN,NaN,When the young people returned to the ballroom...,-0.340259,0.464009
1,85aa80a4c,NaN,NaN,"All through dinner time, Mrs. Fayre was somewh...",-0.315372,0.480805
2,b69ac6792,NaN,NaN,"As Roger had predicted, the snow departed as q...",-0.580118,0.476676
3,dd1000b26,NaN,NaN,And outside before the palace a great garden w...,-1.054013,0.450007
4,37c1b32fb,NaN,NaN,Once upon a time there were Three Bears who li...,0.247197,0.510845
...,...,...,...,...,...,...
2828,25ca8f498,https://sites.ehe.osu.edu/beyondpenguins/files...,CC BY-SA 3.0,When you think of dinosaurs and where they liv...,1.711390,0.646900
2829,2c26db523,https://en.wikibooks.org/wiki/Wikijunior:The_E...,CC BY-SA 3.0,So what is a solid? Solids are usually hard be...,0.189476,0.535648
2830,cd19e2350,https://en.wikibooks.org/wiki/Wikijunior:The_E...,CC BY-SA 3.0,The second state of matter we will discuss is ...,0.255209,0.483866
2831,15e2e9e7a,https://en.wikibooks.org/wiki/Geometry_for_Ele...,CC BY-SA 3.0,Solids are shapes that you can actually touch....,-0.215279,0.514128


### Config and Seeding

In [6]:
class Config(): 
    NUM_FOLDS = 6
    NUM_EPOCHS = 3
    BATCH_SIZE = 16
    MAX_LEN = 248
    EVAL_SCHEDULE = [(0.50, 16), (0.49, 8), (0.48, 4), (0.47, 2), (-1., 1)]
    ROBERTA_PATH = str(MODELS_PATH/'roberta-base_lm')
    TOKENIZER_PATH = str(MODELS_PATH/'roberta-base-0')
    DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
    SEED = 1000
    NUM_WORKERS = 2
    MODEL_FOLDER = MODELS_PATH
    model_name = 'roberta-base'
    svm_kernels = ['rbf']
    svm_c = 5

cfg = Config()

In [7]:
if not cfg.MODEL_FOLDER.exists():
    os.mkdir(cfg.MODEL_FOLDER)

In [8]:
def set_random_seed(random_seed):
    random.seed(random_seed)
    np.random.seed(random_seed)
    os.environ["PYTHONHASHSEED"] = str(random_seed)

    torch.manual_seed(random_seed)
    torch.cuda.manual_seed(random_seed)
    torch.cuda.manual_seed_all(random_seed)

    torch.backends.cudnn.deterministic = True

### Dataset

In [28]:
def add_bins(train_df, num_bins):
    train_df.loc[:, 'bins'] = pd.cut(train_df['target'], bins=num_bins, labels=False)
    return num_bins

In [29]:
add_bins(train_df, cfg.NUM_FOLDS)

6

In [33]:
train_df.groupby(['bins'])['target'].agg(['count', 'mean'])

,count,mean
bins,,
0,122,-3.125765
1,441,-2.270279
2,784,-1.412150
3,886,-0.548095
4,494,0.289716
5,106,1.070237


In [34]:
tokenizer = AutoTokenizer.from_pretrained(cfg.TOKENIZER_PATH)

In [51]:
class CommonLitDataset(Dataset):
    def __init__(self, df, tokenizer, inference_only=False):
        super().__init__()
        self.df, self.inference_only = df, inference_only
        self.text = df['excerpt'].tolist()
        self.bins = df['bins']
        if not inference_only:
            self.target = torch.tensor(df['target'].to_numpy(), dtype = torch.float32)
        
        self.encoded = tokenizer.batch_encode_plus(
            self.text,
            padding = 'max_length',
            max_length = cfg.MAX_LEN,
            truncation = True,
            return_attention_mask=True
        )
        
    def __getitem__(self, index):        
        input_ids = torch.tensor(self.encoded['input_ids'][index])
        attention_mask = torch.tensor(self.encoded['attention_mask'][index])
        
        if self.inference_only:
            return {'input_ids': input_ids, 'attention_mask': attention_mask}
        else:
            target = self.target[index]
            return {'input_ids': input_ids, 'attention_mask': attention_mask, 'target': target}
    
    def __len__(self):
        return len(self.df)

In [52]:
sample_ds = CommonLitDataset(train_df, tokenizer)

### Model

In [74]:
class AttentionHead(nn.Module):
    
    def __init__(self, in_features, hidden_dim, num_targets):
        super().__init__()
        self.in_features = in_features
        
        self.hidden_layer = nn.Linear(in_features, hidden_dim)
        self.final_layer = nn.Linear(hidden_dim, num_targets)
        self.out_features = hidden_dim
        
    def forward(self, features):
        att = torch.tanh(self.hidden_layer(features))
        score = self.final_layer(att)
        attention_weights = torch.softmax(score, dim=1)
        return attention_weights

In [75]:
class CommonLitModel(nn.Module):
    def __init__(self):
        super(CommonLitModel, self).__init__()
        config = AutoConfig.from_pretrained(cfg.ROBERTA_PATH)
        config.update({
            "output_hidden_states": True,
            "hidden_dropout_prob": 0.0,
            "layer_norm_eps": 1e-7
        })
        self.transformer_model = AutoModel.from_pretrained(cfg.ROBERTA_PATH, config=config)
        self.attention = AttentionHead(config.hidden_size, 512, 1)
        self.regressor = nn.Linear(config.hidden_size, 1)
    
    def forward(self, input_ids, attention_mask):
        last_layer_hidden_states = self.transformer_model(input_ids=input_ids, attention_mask=attention_mask)['last_hidden_state']
        weights = self.attention(last_layer_hidden_states)
        context_vector = torch.sum(weights * last_layer_hidden_states, dim=1) 
        return self.regressor(context_vector), context_vector

In [55]:
sample_model = CommonLitModel()

Some weights of the model checkpoint at /home/commonlit/models/roberta-base_lm were not used when initializing RobertaModel: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at /home/commonlit/models/roberta-base_lm and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a

In [116]:
a = torch.randn(8, 256, 768)

In [121]:
torch.sum((sample_model.attention(a) * a), dim=1).shape

torch.Size([8, 768])

### Evaluation and Prediction

In [56]:
def eval_mse(model, data_loader):
    model.eval()
    mse_sum = 0
    mse_loss = nn.MSELoss(reduction='sum')
    
    with torch.no_grad():
        for batch_num, record in enumerate(data_loader):
            input_ids, attention_mask, target = record['input_ids'].to(cfg.DEVICE), record['attention_mask'].to(cfg.DEVICE), record['target'].to(cfg.DEVICE)
            pred = model(input_ids, attention_mask)
            mse_sum += mse_loss(pred.flatten().cpu(), target.cpu())
            
    return mse_sum / len(data_loader.dataset)

In [57]:
def predict(model, data_loader):
    model.eval()
    result = []
    
    with torch.no_grad():
        for batch_num, record in tqdm(enumerate(data_loader), total=len(data_loader)):
            input_ids, attention_mask = record['input_ids'].to(cfg.DEVICE), record['attention_mask'].to(cfg.DEVICE)
            pred = model(input_ids, attention_mask)
            result.extend(pred.flatten().to("cpu").tolist())
            
    return np.array(result)

In [58]:
sample_dl = DataLoader(sample_ds, shuffle=False, batch_size=16, num_workers=1)

### Optimizer and Sampler

In [59]:
def create_optimizer(model, base_lr=5e-5):
    named_parameters = list(model.named_parameters())
    
    regressor_param_start = 203
    roberta_parameters = named_parameters[:197]
    attention_parameters = named_parameters[199:regressor_param_start]
    regressor_parameters = named_parameters[regressor_param_start:]
    
    attention_group = [params for (name, params) in attention_parameters]
    regressor_group = [params for (name, params) in regressor_parameters]
    
    parameters = []
    parameters.append({"params": attention_group})
    parameters.append({"params": regressor_group})
    
    for layer_num, (name, params) in enumerate(roberta_parameters):
        weight_decay = 0.0 if 'bias' in name else 0.01
        
        lr = base_lr / 2.5
        if layer_num >= 133:
            lr = base_lr / 0.5
        elif layer_num >= 69:        
            lr = base_lr    
            
        parameters.append({"params": params,
                           "weight_decay": weight_decay,
                           "lr": lr})
        
    return AdamW(parameters)

In [60]:
sample_optimizer = create_optimizer(sample_model)

In [61]:
from torch.utils.data import Sampler,SequentialSampler,RandomSampler,SubsetRandomSampler
from collections import Counter

class WeightedSampler(Sampler):
    
    def __init__(self, dataset):
        
        self.indices = list(range(len(dataset)))
        self.num_samples = len(dataset)
        self.label_to_count = dict(Counter(dataset.bins))
        weights = [1/self.label_to_count[i] for i in dataset.bins]
        
        self.weights = torch.tensor(weights,dtype=torch.double)
        
    def __iter__(self):
        count = 0
        index = [self.indices[i] for i in torch.multinomial(self.weights, self.num_samples, replacement=True)]
        while count < self.num_samples:
            yield index[count]
            count += 1
    
    def __len__(self):
        return self.num_samples

### Training

In [62]:
def choose_eval_period(val_rmse):
    for rmse, period in cfg.EVAL_SCHEDULE:
        if val_rmse >= rmse:
            return period

In [63]:
def serialize_best(best_val_rmse, best_epoch, val_rmse, epoch, model, model_path):
    if not best_val_rmse or val_rmse < best_val_rmse:
        best_val_rmse = val_rmse
        best_epoch = epoch
        if not model_path.parent.exists():
            os.makedirs(model_path.parent)
        
        torch.save(model.state_dict(), model_path)
        print(f"New best_val_rmse: {best_val_rmse:0.4}")
    else:       
        print(f"Still best_val_rmse: {best_val_rmse:0.4}",
              f"(from epoch {best_epoch})")
    return best_epoch, best_val_rmse

In [64]:
class Trainer():
    def __init__(self, model, model_path, train_loader, val_loader, optimizer, scheduler=None, num_epochs=cfg.NUM_EPOCHS):
        self.model, self.model_path, self.train_loader, self.val_loader, self.optimizer, self.scheduler, self.num_epochs = (
            model, model_path, train_loader, val_loader, optimizer, scheduler, num_epochs
        )
            
    def train(self):
        self.model.train()
        mse_loss = nn.MSELoss(reduction='mean')
        
        best_val_rmse = None
        best_epoch = 0
        step = 0
        last_eval_step = 0
        eval_period = cfg.EVAL_SCHEDULE[0][1]    

        start = time.time()
        
        tbar = tqdm(range(self.num_epochs), total=self.num_epochs)
        for epoch in tbar:
            tbar.set_description(f'Epoch: {epoch}')
            val_rmse = None
            for batch_num, record in enumerate(self.train_loader):
                input_ids, attention_mask, target = record['input_ids'].to(cfg.DEVICE), record['attention_mask'].to(cfg.DEVICE), record['target'].to(cfg.DEVICE)
                
                self.optimizer.zero_grad()
                
                pred = self.model(input_ids, attention_mask)
                
                mse = mse_loss(pred.flatten(), target)
                
                mse.backward()
                
                self.optimizer.step()
                if self.scheduler:
                    self.scheduler.step()
                    
                if step >= last_eval_step + eval_period:
                    elapsed_seconds = time.time() - start
                    num_steps = step - last_eval_step
                    print(f"\n{num_steps} steps took {elapsed_seconds:0.3} seconds")
                    last_eval_step = step
                    
                    val_rmse = np.sqrt(eval_mse(self.model, self.val_loader))
                    print(f"Epoch: {epoch} batch_num: {batch_num}", f"val_rmse: {val_rmse:0.4} ", end='')
                    
                    eval_period = choose_eval_period(val_rmse)
                    best_epoch, best_val_rmse = serialize_best(best_val_rmse, best_epoch, val_rmse, epoch, model, self.model_path)
                    start = time.time()
                
                step += 1
        return best_val_rmse

In [65]:
kfold = KFold(n_splits=cfg.NUM_FOLDS, random_state=cfg.SEED, shuffle=True)
splits = kfold.split(train_df)

In [67]:
list_val_rmse = []

pbar = tqdm(enumerate(splits), total=cfg.NUM_FOLDS, position=0, leave=True)
for fold, (train_indices, val_indices) in pbar:
    pbar.set_description(f'Fold {fold}')
    model_path = cfg.MODEL_FOLDER/f"roberta-base_{fold + 1}/model_{fold + 1}.pth"
    
    set_random_seed(cfg.SEED + fold)
    
    tokenizer = AutoTokenizer.from_pretrained(cfg.TOKENIZER_PATH)
    
    train_dataset = CommonLitDataset(train_df.loc[train_indices], tokenizer)    
    val_dataset = CommonLitDataset(train_df.loc[val_indices], tokenizer)
    
    train_loader = DataLoader(train_dataset, sampler=WeightedSampler(train_dataset), batch_size=cfg.BATCH_SIZE,
                              drop_last=False, shuffle=False, num_workers=cfg.NUM_WORKERS)    
    val_loader = DataLoader(val_dataset, batch_size=cfg.BATCH_SIZE,
                            drop_last=False, shuffle=False, num_workers=cfg.NUM_WORKERS)
    
    set_random_seed(cfg.SEED + fold)
    
    model = CommonLitModel().to(cfg.DEVICE)
        
    optimizer = create_optimizer(model)
    
    scheduler = get_cosine_schedule_with_warmup(optimizer,
                                                num_training_steps=cfg.NUM_EPOCHS * len(train_loader), 
                                                num_warmup_steps=50)
    
    trainer = Trainer(model, model_path, train_loader, val_loader, optimizer, scheduler = scheduler)
    list_val_rmse.append(trainer.train())
    
    tokenizer.save_pretrained(str(model_path.parent))
    del model
    gc.collect()
    
    if cfg.DEVICE == 'cuda':
        torch.cuda.empty_cache()
        
print("\nPerformance estimates:")
print(list_val_rmse)
print("Mean:", np.array(list_val_rmse).mean())

Some weights of the model checkpoint at /home/commonlit/models/roberta-base_lm were not used when initializing RobertaModel: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at /home/commonlit/models/roberta-base_lm and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a


16 steps took 5.01 seconds
Epoch: 0 batch_num: 16 val_rmse: 0.9504 New best_val_rmse: 0.9504

16 steps took 3.8 seconds
Epoch: 0 batch_num: 32 val_rmse: 0.7897 New best_val_rmse: 0.7897

16 steps took 3.81 seconds
Epoch: 0 batch_num: 48 val_rmse: 0.7539 New best_val_rmse: 0.7539

16 steps took 3.79 seconds
Epoch: 0 batch_num: 64 val_rmse: 0.6913 New best_val_rmse: 0.6913

16 steps took 3.81 seconds
Epoch: 0 batch_num: 80 val_rmse: 0.5913 New best_val_rmse: 0.5913

16 steps took 3.86 seconds
Epoch: 0 batch_num: 96 val_rmse: 0.5479 New best_val_rmse: 0.5479

16 steps took 3.87 seconds
Epoch: 0 batch_num: 112 val_rmse: 0.5653 Still best_val_rmse: 0.5479 (from epoch 0)

16 steps took 3.85 seconds
Epoch: 0 batch_num: 128 val_rmse: 0.6806 Still best_val_rmse: 0.5479 (from epoch 0)

16 steps took 3.85 seconds
Epoch: 0 batch_num: 144 val_rmse: 0.5339 New best_val_rmse: 0.5339

16 steps took 4.31 seconds
Epoch: 1 batch_num: 12 val_rmse: 0.5389 Still best_val_rmse: 0.5339 (from epoch 0)

16 ste

Some weights of the model checkpoint at /home/commonlit/models/roberta-base_lm were not used when initializing RobertaModel: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at /home/commonlit/models/roberta-base_lm and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a


16 steps took 4.67 seconds
Epoch: 0 batch_num: 16 val_rmse: 1.047 New best_val_rmse: 1.047

16 steps took 3.86 seconds
Epoch: 0 batch_num: 32 val_rmse: 0.7213 New best_val_rmse: 0.7213

16 steps took 3.94 seconds
Epoch: 0 batch_num: 48 val_rmse: 0.684 New best_val_rmse: 0.684

16 steps took 3.91 seconds
Epoch: 0 batch_num: 64 val_rmse: 0.6205 New best_val_rmse: 0.6205

16 steps took 3.94 seconds
Epoch: 0 batch_num: 80 val_rmse: 0.5899 New best_val_rmse: 0.5899

16 steps took 3.92 seconds
Epoch: 0 batch_num: 96 val_rmse: 0.6759 Still best_val_rmse: 0.5899 (from epoch 0)

16 steps took 3.89 seconds
Epoch: 0 batch_num: 112 val_rmse: 0.6428 Still best_val_rmse: 0.5899 (from epoch 0)

16 steps took 3.94 seconds
Epoch: 0 batch_num: 128 val_rmse: 0.5893 New best_val_rmse: 0.5893

16 steps took 3.93 seconds
Epoch: 0 batch_num: 144 val_rmse: 0.5338 New best_val_rmse: 0.5338

16 steps took 4.18 seconds
Epoch: 1 batch_num: 12 val_rmse: 0.5349 Still best_val_rmse: 0.5338 (from epoch 0)

16 steps 

Some weights of the model checkpoint at /home/commonlit/models/roberta-base_lm were not used when initializing RobertaModel: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at /home/commonlit/models/roberta-base_lm and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a


16 steps took 4.64 seconds
Epoch: 0 batch_num: 16 val_rmse: 0.8523 New best_val_rmse: 0.8523

16 steps took 3.91 seconds
Epoch: 0 batch_num: 32 val_rmse: 0.9049 Still best_val_rmse: 0.8523 (from epoch 0)

16 steps took 3.87 seconds
Epoch: 0 batch_num: 48 val_rmse: 0.7095 New best_val_rmse: 0.7095

16 steps took 3.87 seconds
Epoch: 0 batch_num: 64 val_rmse: 0.686 New best_val_rmse: 0.686

16 steps took 3.91 seconds
Epoch: 0 batch_num: 80 val_rmse: 0.7221 Still best_val_rmse: 0.686 (from epoch 0)

16 steps took 3.87 seconds
Epoch: 0 batch_num: 96 val_rmse: 0.6423 New best_val_rmse: 0.6423

16 steps took 3.87 seconds
Epoch: 0 batch_num: 112 val_rmse: 0.5901 New best_val_rmse: 0.5901

16 steps took 3.91 seconds
Epoch: 0 batch_num: 128 val_rmse: 0.5677 New best_val_rmse: 0.5677

16 steps took 3.87 seconds
Epoch: 0 batch_num: 144 val_rmse: 0.5743 Still best_val_rmse: 0.5677 (from epoch 0)

16 steps took 4.24 seconds
Epoch: 1 batch_num: 12 val_rmse: 0.5626 New best_val_rmse: 0.5626

16 steps

Some weights of the model checkpoint at /home/commonlit/models/roberta-base_lm were not used when initializing RobertaModel: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at /home/commonlit/models/roberta-base_lm and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a


16 steps took 4.62 seconds
Epoch: 0 batch_num: 16 val_rmse: 1.076 New best_val_rmse: 1.076

16 steps took 3.89 seconds
Epoch: 0 batch_num: 32 val_rmse: 0.7772 New best_val_rmse: 0.7772

16 steps took 3.9 seconds
Epoch: 0 batch_num: 48 val_rmse: 0.6755 New best_val_rmse: 0.6755

16 steps took 3.92 seconds
Epoch: 0 batch_num: 64 val_rmse: 0.7292 Still best_val_rmse: 0.6755 (from epoch 0)

16 steps took 3.87 seconds
Epoch: 0 batch_num: 80 val_rmse: 0.6183 New best_val_rmse: 0.6183

16 steps took 3.9 seconds
Epoch: 0 batch_num: 96 val_rmse: 0.5755 New best_val_rmse: 0.5755

16 steps took 3.91 seconds
Epoch: 0 batch_num: 112 val_rmse: 0.5493 New best_val_rmse: 0.5493

16 steps took 3.91 seconds
Epoch: 0 batch_num: 128 val_rmse: 0.5339 New best_val_rmse: 0.5339

16 steps took 3.91 seconds
Epoch: 0 batch_num: 144 val_rmse: 0.6544 Still best_val_rmse: 0.5339 (from epoch 0)

16 steps took 4.15 seconds
Epoch: 1 batch_num: 12 val_rmse: 0.5758 Still best_val_rmse: 0.5339 (from epoch 0)

16 steps 

Some weights of the model checkpoint at /home/commonlit/models/roberta-base_lm were not used when initializing RobertaModel: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at /home/commonlit/models/roberta-base_lm and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a


16 steps took 4.54 seconds
Epoch: 0 batch_num: 16 val_rmse: 0.8104 New best_val_rmse: 0.8104

16 steps took 3.86 seconds
Epoch: 0 batch_num: 32 val_rmse: 1.259 Still best_val_rmse: 0.8104 (from epoch 0)

16 steps took 3.84 seconds
Epoch: 0 batch_num: 48 val_rmse: 0.7261 New best_val_rmse: 0.7261

16 steps took 3.91 seconds
Epoch: 0 batch_num: 64 val_rmse: 0.7352 Still best_val_rmse: 0.7261 (from epoch 0)

16 steps took 3.93 seconds
Epoch: 0 batch_num: 80 val_rmse: 0.6073 New best_val_rmse: 0.6073

16 steps took 3.85 seconds
Epoch: 0 batch_num: 96 val_rmse: 0.6192 Still best_val_rmse: 0.6073 (from epoch 0)

16 steps took 3.93 seconds
Epoch: 0 batch_num: 112 val_rmse: 0.5488 New best_val_rmse: 0.5488

16 steps took 3.89 seconds
Epoch: 0 batch_num: 128 val_rmse: 0.6277 Still best_val_rmse: 0.5488 (from epoch 0)

16 steps took 3.9 seconds
Epoch: 0 batch_num: 144 val_rmse: 0.5508 Still best_val_rmse: 0.5488 (from epoch 0)

16 steps took 4.29 seconds
Epoch: 1 batch_num: 12 val_rmse: 0.6197 

### Verify the model

In [68]:
from sklearn.svm import SVR
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import mean_squared_error
from tqdm.notebook import tqdm

In [69]:
cfg.model_offset = 0
cfg.model_limit = 6
cfg.n_folds = 5
cfg.svm_kernels = ['rbf']
cfg.svm_c = 5

In [70]:
num_bins = int(np.ceil(np.log2(len(train_df))))
train_df['bins'] = pd.cut(train_df['target'], bins=num_bins, labels=False)
bins = train_df['bins'].values

In [76]:
%%time

inference_models = []
for i in range(1, cfg.NUM_FOLDS + 1):
    print(f'Model {i}')
    inference_model = CommonLitModel()
    inference_model = inference_model.cuda()
    inference_model.load_state_dict(torch.load(str(MODELS_PATH/f"{cfg.model_name.replace('/', '_')}_{i}/model_{i}.pth")))
    inference_model.eval();
    inference_models.append(inference_model)

Model 1


Some weights of the model checkpoint at /home/commonlit/models/roberta-base_lm were not used when initializing RobertaModel: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at /home/commonlit/models/roberta-base_lm and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a

Model 2


Some weights of the model checkpoint at /home/commonlit/models/roberta-base_lm were not used when initializing RobertaModel: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at /home/commonlit/models/roberta-base_lm and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a

Model 3


...

Model 5


Some weights of the model checkpoint at /home/commonlit/models/roberta-base_lm were not used when initializing RobertaModel: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at /home/commonlit/models/roberta-base_lm and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a

Model 6


Some weights of the model checkpoint at /home/commonlit/models/roberta-base_lm were not used when initializing RobertaModel: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at /home/commonlit/models/roberta-base_lm and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a

CPU times: user 7.66 s, sys: 2.64 s, total: 10.3 s
Wall time: 8.63 s


In [77]:
tokenizers = []
for i in range(1, cfg.NUM_FOLDS):
    tokenizer = AutoTokenizer.from_pretrained(MODELS_PATH/f"{cfg.model_name.replace('/', '_')}-{i}")
    tokenizers.append(tokenizer)

In [78]:
def get_cls_embeddings(dl, transformer_model):
    cls_embeddings = []
    with torch.no_grad():
        for input_features in tqdm(dl, total=len(dl)):
            output, context_vector = transformer_model(input_features['input_ids'].cuda(), input_features['attention_mask'].cuda())
#             cls_embeddings.extend(output['last_hidden_state'][:,0,:].detach().cpu().numpy())
            embedding_out = context_vector.detach().cpu().numpy()
            cls_embeddings.extend(embedding_out)
    return np.array(cls_embeddings)

In [79]:
def rmse_score(X, y):
    return np.sqrt(mean_squared_error(X, y))

In [80]:
def convert_to_list(t):
    return t.flatten().long()

class CommonLitDataset(nn.Module):
    def __init__(self, text, test_id, tokenizer, max_len=128):
        self.excerpt = text
        self.test_id = test_id
        self.max_len = max_len
        self.tokenizer = tokenizer
    
    def __getitem__(self,idx):
        encode = self.tokenizer(self.excerpt[idx],
                                return_tensors='pt',
                                max_length=self.max_len,
                                padding='max_length',
                                truncation=True)
        return {'input_ids': convert_to_list(encode['input_ids']),
                'attention_mask': convert_to_list(encode['attention_mask']),
                'id': self.test_id[idx]}
    
    def __len__(self):
        return len(self.excerpt)

In [81]:
def create_dl(df, tokenizer):
    text = df['excerpt'].values
    ids = df['id'].values
    ds = CommonLitDataset(text, ids, tokenizer, max_len=cfg.MAX_LEN)
    return DataLoader(ds, 
                      batch_size = cfg.BATCH_SIZE,
                      shuffle=False,
                      num_workers = 1,
                      pin_memory=True,
                      drop_last=False
                     )

In [82]:
train_df = pd.read_csv(DATA_PATH/'train-orig.csv')
test_df = pd.read_csv(DATA_PATH/'test.csv')
remove_unnecessary(train_df)

In [83]:
train_target_mean = train_df['target'].mean()
train_target_std = train_df['target'].std()
train_df['normalized_target'] = (train_df['target'] - train_target_mean) / train_target_std

In [85]:
%%time

train_target = train_df['normalized_target'].values

def calc_mean(scores):
    return np.mean(np.array(scores), axis=0)

final_scores = []
final_rmse = []
kernel_rmse_score_mean = []
final_kernel_predictions_means = []
for j, (inference_model, tokenizer) in enumerate(zip(inference_models, tokenizers)):
    print('Model', j)
    test_dl = create_dl(test_df, tokenizer)
    train_dl = create_dl(train_df, tokenizer)
    transformer_model = inference_model
    transformer_model.cuda()
    X = get_cls_embeddings(train_dl, transformer_model)
    
    y = train_target
    X_test = get_cls_embeddings(test_dl, transformer_model)
    
    kfold = StratifiedKFold(n_splits=cfg.NUM_FOLDS)
    scores = []
    rmse_scores = []
    kernel_predictions_means = []
    for kernel in cfg.svm_kernels:
        print('Kernel', kernel)
        kernel_scores = []
        kernel_rmse_scores = []
        kernel_predictions = []
        for k, (train_idx, valid_idx) in enumerate(kfold.split(X, bins)):

            print('Fold', k, train_idx.shape, valid_idx.shape)
            model = SVR(C=cfg.svm_c, kernel=kernel, gamma='auto')

            X_train, y_train = X[train_idx], y[train_idx]
            X_valid, y_valid = X[valid_idx], y[valid_idx]
            model.fit(X_train, y_train)
            prediction = model.predict(X_valid)
            kernel_predictions.append(prediction)
            kernel_rmse_scores.append(rmse_score(prediction, y_valid))
            print('rmse_score', kernel_rmse_scores[k])
            kernel_scores.append(model.predict(X_test))
        kernel_predictions_means.append(np.array([np.mean(kp) for kp in kernel_predictions]).mean())
        scores.append(calc_mean(kernel_scores))
        kernel_rmse_score = calc_mean(kernel_rmse_scores)
        kernel_rmse_score_mean.append(kernel_rmse_score)
        rmse_scores.append(kernel_rmse_score)
    final_kernel_predictions_means.append(kernel_predictions_means)
    final_scores.append(calc_mean(scores))
    final_rmse.append(calc_mean(rmse_scores))
print('FINAL RMSE score', np.mean(np.array(final_rmse)))

Model 0



Kernel rbf
Fold 0 (2360,) (473,)
rmse_score 0.312142841484001
Fold 1 (2361,) (472,)
rmse_score 0.35453536052747725
Fold 2 (2361,) (472,)
rmse_score 0.32059224740411746
Fold 3 (2361,) (472,)
rmse_score 0.3342262121475011
Fold 4 (2361,) (472,)
rmse_score 0.33193625064639976
Fold 5 (2361,) (472,)
rmse_score 0.3588098979584189
Model 1



Kernel rbf
Fold 0 (2360,) (473,)
rmse_score 0.3314009215674823
Fold 1 (2361,) (472,)
rmse_score 0.33989511856480586
Fold 2 (2361,) (472,)
rmse_score 0.3200691621809814
Fold 3 (2361,) (472,)
rmse_score 0.30714172510842125
Fold 4 (2361,) (472,)
rmse_score 0.3297242850958922
Fold 5 (2361,) (472,)
rmse_score 0.3351967107631785
Model 2



Kernel rbf
Fold 0 (2360,) (473,)
rmse_score 0.31492303382301645
Fold 1 (2361,) (472,)
rmse_score 0.3710494337479098
Fold 2 (2361,) (472,)
rmse_score 0.3354141274363938
Fold 3 (2361,) (472,)
rmse_score 0.3243960011506718
Fold 4 (2361,) (472,)
rmse_score 0.3308837801548399
Fold 5 (2361,) (472,)
rmse_score 0.3435653199727582
Model 3



Kernel rbf
Fold 0 (2360,) (473,)
rmse_score 0.33738823237280635
Fold 1 (2361,) (472,)
rmse_score 0.3421355049648379
Fold 2 (2361,) (472,)
rmse_score 0.316694350146067
Fold 3 (2361,) (472,)
rmse_score 0.3034172157657744
Fold 4 (2361,) (472,)
rmse_score 0.35079517899032514
Fold 5 (2361,) (472,)
rmse_score 0.3670471707717054
Model 4



Kernel rbf
Fold 0 (2360,) (473,)
rmse_score 0.345575946808507
Fold 1 (2361,) (472,)
rmse_score 0.3622680371661247
Fold 2 (2361,) (472,)
rmse_score 0.34982519235186066
Fold 3 (2361,) (472,)
rmse_score 0.3359857872447713
Fold 4 (2361,) (472,)
rmse_score 0.3384502199006281
Fold 5 (2361,) (472,)
rmse_score 0.37340757962166077
FINAL RMSE score 0.33729642819464456
CPU times: user 2min 8s, sys: 2.92 s, total: 2min 11s
Wall time: 2min 10s


In [86]:
final_kernel_predictions_means

[[-0.004122795950432596],
 [-0.0008061889274363845],
 [0.0018296439835301104],
 [-0.0015377556223151255],
 [-0.0023664210936125934]]

In [87]:
# (train_df['target'] - cfg.train_target_mean) / cfg.train_target_std
final_scores_normalized = np.array(final_scores) * train_target_std + train_target_mean

In [88]:
kernel_rmse_score_mean_array = np.array(kernel_rmse_score_mean)
kernel_rmse_score_mean_sum = np.sum(kernel_rmse_score_mean_array)
prop_losses = kernel_rmse_score_mean_array / kernel_rmse_score_mean_sum
prop_losses_sum = (1 - prop_losses).sum()
weights = (1 - prop_losses) / prop_losses_sum
weights

array([0.20028501, 0.20149104, 0.20008763, 0.20015567, 0.19798065])

In [89]:
def calc_mean(scores, weights=weights):
    return np.average(np.array(scores), weights=weights, axis=0)

In [90]:
target_mean = train_df['target'].mean()
final_scores_flat = calc_mean(final_scores_normalized).flatten()
final_scores_mean = final_scores_flat.mean()
target_mean, np.array(final_scores_normalized).mean()
# (-0.9579984513405823, -0.8029817438292849)

(-0.9596573929279916, -0.9767934966056343)

In [91]:
final_scores_flat

array([-0.54467689, -0.46206632, -0.38874319, -2.42437832, -1.85041837,
       -1.60337413,  0.43547454])

In [93]:
mean_diff = target_mean - final_scores_mean
mean_diff, mean_diff / len(final_scores)

(0.01722584639091085, 0.0034451692781821697)

In [94]:
sample_df['target'] = final_scores_flat + mean_diff
# sample_df['target'] = len(final_scores) / np.sum(1 / np.array(final_scores), axis=0) # harmonic mean
sample_df

,id,target
0,c0f722661,-0.527451
1,f0953f0a5,-0.444840
2,0df072751,-0.371517
3,04caf4e0c,-2.407152
4,0e63f8bea,-1.833193
5,12537fe78,-1.586148
6,965e592c0,0.452700


### Prepare Packaging

In [95]:
cfg.model_name

'roberta-base'

In [96]:
BEST_MODEL_FOLDER = MODELS_PATH/cfg.model_name/'best'
!rm -rf {BEST_MODEL_FOLDER}
!mkdir -p {BEST_MODEL_FOLDER}

In [97]:
BEST_MODEL_FOLDER

PosixPath('/home/commonlit/models/roberta-base/best')

In [98]:
cfg.NUM_FOLDS

6

In [99]:
bestmodels = [MODELS_PATH/f'{cfg.model_name}_{i + 1}' for i in range(0, cfg.NUM_FOLDS)]

In [100]:
bestmodels

[PosixPath('/home/commonlit/models/roberta-base_1'),
 PosixPath('/home/commonlit/models/roberta-base_2'),
 PosixPath('/home/commonlit/models/roberta-base_3'),
 PosixPath('/home/commonlit/models/roberta-base_4'),
 PosixPath('/home/commonlit/models/roberta-base_5'),
 PosixPath('/home/commonlit/models/roberta-base_6')]

In [101]:
from shutil import copyfile

def normalize_name(path_name):
    return path_name.replace('', '')

for i, best_model in enumerate(bestmodels):
    print(f'Processing {i}th model')
    best_model_file = f'{best_model}/model_{i + 1}.pth'
    if Path(best_model_file).exists():
        copyfile(best_model_file, f'{BEST_MODEL_FOLDER}/{i}_pytorch_model.bin')
        tokenizer_path = Path(BEST_MODEL_FOLDER/f'tokenizer-{i}')
        tokenizer_path.mkdir(parents=True, exist_ok=True)
        assert tokenizer_path.exists()

        tokenizer_json = Path(normalize_name(f'{MODELS_PATH/cfg.model_name}-{i}/tokenizer.json'))
        assert tokenizer_json.exists(), f'{tokenizer_json} does not exist'
        copyfile(tokenizer_json, tokenizer_path/'tokenizer.json')

        vocab_txt = Path(normalize_name(f'{MODELS_PATH/cfg.model_name}-{i}/vocab.json'))
        assert vocab_txt.exists(), f'{vocab_txt} does not exist'
        copyfile(vocab_txt, tokenizer_path/'vocab.json')

        config_json = Path(normalize_name(f'{MODELS_PATH/cfg.model_name}-{i}/config.json'))
        assert config_json.exists()
        copyfile(config_json, tokenizer_path/'config.json')
    else:
        print(f'{best_model_file} is missing')

Processing 0th model
Processing 1th model
Processing 2th model
Processing 3th model
Processing 4th model
Processing 5th model


In [102]:
import shutil

shutil.make_archive(MODELS_PATH/cfg.model_name/'best_models', 'zip', BEST_MODEL_FOLDER)

'/home/commonlit/models/roberta-base/best_models.zip'

In [103]:
!ls {MODELS_PATH/cfg.model_name}

best  best_models.zip  dataset-metadata.json  lm.zip  roberta-base.yaml


In [104]:
!mv {MODELS_PATH}/{cfg.model_name}.yaml {MODELS_PATH/cfg.model_name}

mv: cannot stat '/home/commonlit/models/roberta-base.yaml': No such file or directory


In [106]:
transformer_model.transformer_model.save_pretrained(save_directory=f'{MODELS_PATH/cfg.model_name}/lm')

In [107]:
!du -h {MODELS_PATH/cfg.model_name}/*

2.1M	/home/commonlit/models/roberta-base/best/tokenizer-0
2.1M	/home/commonlit/models/roberta-base/best/tokenizer-1
2.1M	/home/commonlit/models/roberta-base/best/tokenizer-2
2.1M	/home/commonlit/models/roberta-base/best/tokenizer-3
2.1M	/home/commonlit/models/roberta-base/best/tokenizer-4
2.1M	/home/commonlit/models/roberta-base/best/tokenizer-5
2.9G	/home/commonlit/models/roberta-base/best
2.7G	/home/commonlit/models/roberta-base/best_models.zip
4.0K	/home/commonlit/models/roberta-base/dataset-metadata.json
476M	/home/commonlit/models/roberta-base/lm
442M	/home/commonlit/models/roberta-base/lm.zip
4.0K	/home/commonlit/models/roberta-base/roberta-base.yaml


In [108]:
shutil.make_archive(MODELS_PATH/cfg.model_name/'lm', 'zip', f'{MODELS_PATH/cfg.model_name}/lm')

'/home/commonlit/models/roberta-base/lm.zip'

In [109]:
!kaggle datasets init -p {MODELS_PATH/cfg.model_name}

Data package template written to: /home/commonlit/models/roberta-base/dataset-metadata.json


In [110]:
dataset_json_path = Path(MODELS_PATH/cfg.model_name/'dataset-metadata.json')
assert dataset_json_path.exists()

In [111]:
!cat {str(dataset_json_path)}

{
  "title": "INSERT_TITLE_HERE",
  "id": "gilfernandes/INSERT_SLUG_HERE",
  "licenses": [
    {
      "name": "CC0-1.0"
    }
  ]
}

In [112]:
with open(dataset_json_path, 'r') as f:
    dataset_json = f.read()
    dataset_json = dataset_json.replace('INSERT_TITLE_HERE', f'commonlit-{cfg.model_name}-light-balanced').replace('INSERT_SLUG_HERE', f'commonlit-{cfg.model_name}-light-balanced')
    print(dataset_json)
with(open(dataset_json_path, 'w')) as f:
    f.write(dataset_json)

{
  "title": "commonlit-roberta-base-light-balanced",
  "id": "gilfernandes/commonlit-roberta-base-light-balanced",
  "licenses": [
    {
      "name": "CC0-1.0"
    }
  ]
}


In [113]:
!rm -rf {MODELS_PATH/cfg.model_name}/best
!rm -rf {MODELS_PATH/cfg.model_name}/lm

In [114]:
!kaggle datasets create -p {MODELS_PATH/cfg.model_name}

Starting upload for file best_models.zip
100%|██████████████████████████████████████| 2.60G/2.60G [04:19<00:00, 10.8MB/s]
Upload successful: best_models.zip (3GB)
Starting upload for file roberta-base.yaml
100%|███████████████████████████████████████████| 114/114 [00:07<00:00, 15.1B/s]
Upload successful: roberta-base.yaml (114B)
Starting upload for file lm.zip
100%|████████████████████████████████████████| 442M/442M [00:53<00:00, 8.59MB/s]
Upload successful: lm.zip (442MB)
Your private Dataset is being created. Please check progress at /api/v1/datasets/status//gilfernandes/commonlit-roberta-base-light-balanced


In [ ]:
!kaggle datasets version -p {MODELS_PATH/cfg.model_name} -m "Version with normal distribution by bin and extra pre-training" -d

In [ ]:
state_dict = torch.load(str(MODELS_PATH/f'distilroberta-0/checkpoint-105/pytorch_model.bin'))

In [ ]:
loaded_model = CommonLitModel()

In [ ]:
loaded_model.load_state_dict(state_dict)